# MMDetection 3.0 설치

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Jun 30 03:35:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    45W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# install
!pip3 install openmim
!mim install mmengine
!mim install "mmcv>=2.0.0,<2.1.0"

# gitclone
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -e .

# version
import mmdet
print(mmdet.__version__)
# Example output: 3.0.0, or an another version.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 85.8 MB/s eta 0:00:00
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.3/374.3 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 31.1 MB/s eta 0:00:00
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 MB 24.1 MB/s eta 0:00:00
Cloning into 'mmdetection'...
remote: Enumerating objects: 36658, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 36658 (delta 30), reused 90 (delta 18), pack-reused 36536
Receiving objects: 100% (36658/36658), 56.84 MiB | 11.07 MiB/s, done.
Resolving deltas: 100% (25726/25726), done.
/content/mmdetection
Obtaining file:///content/mmdetection
  Prepar

# 데이터 준비

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# permmision warning 방지
!chmod 600 ~/.kaggle/kaggle.json

# download the data
!kaggle datasets download -d jomainho/snack-data-to50 -p /content/

Saving kaggle.json to kaggle.json
100% 413M/415M [00:21<00:00, 22.3MB/s]
100% 415M/415M [00:21<00:00, 19.9MB/s]


In [ ]:
!unzip -q /content/snack-data-to50.zip -d /content/snack-data-to50

In [ ]:
import json

file_path = '/content/snack-data-to50/train/train.json'

with open(file_path, 'r') as file:
    data = json.load(file)

names = []  # List to store the values related to the 'name' key

# Iterate over the 'annotations' section of the data
for category in data['categories']:
    if 'name' in category:
        name = category['name']
        names.append(name)

# Print the extracted names
print(len(names))
print(names)

50
['65621', '50098', '30152', '45219', '30064', '30166', '50117', '50062', '30120', '20211', '10178', '45221', '10092', '30061', '10091', '30119', '25679', '30086', '50063', '15033', '45222', '65629', '65858', '30066', '50061', '20164', '10094', '90078', '30140', '15046', '45227', '30060', '15175', '10093', '65723', '90072', '10210', '30292', '10209', '45220', '35044', '30099', '30096', '65727', '65719', '65890', '90073', '10095', '20167', '30291']


# 모델 학습

- Configuration 모델 준비

In [ ]:
from mmengine.config import Config
cfg = Config.fromfile('/content/mmdetection/projects/EfficientDet/configs/efficientdet_effb3_bifpn_8xb16-crop896-300e_coco.py')
print(f'Config:\n{cfg.pretty_text}')

Config:
dataset_type = 'CocoDataset'
data_root = 'data/coco/'
backend_args = None
train_pipeline = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='RandomResize',
        scale=(896, 896),
        ratio_range=(0.1, 2.0),
        keep_ratio=True),
    dict(type='RandomCrop', crop_size=(896, 896)),
    dict(type='RandomFlip', prob=0.5),
    dict(type='PackDetInputs')
]
test_pipeline = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(type='Resize', scale=(896, 896), keep_ratio=True),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='PackDetInputs',
        meta_keys=('img_id', 'img_path', 'ori_shape', 'img_shape',
                   'scale_factor'))
]
train_dataloader = dict(
    batch_size=16,
    num_workers=8,
    persistent_workers=True,
    sampler=dict(type='DefaultSampler', shuffle=True, _scope_='mmdet'),
    batch_sampler=dict(type='AspectRatioBatchSampler', _

- Pretrained model 다운로드

In [ ]:
# -c url에서 다운받아 -0 경로에 저장한다.
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmdetection/v3.0/efficientdet/efficientdet_effb3_bifpn_8xb16-crop896-300e_coco/efficientdet_effb3_bifpn_8xb16-crop896-300e_coco_20230223_122457-e6f7a833.pth \
      -O checkpoints/efficientdet_training.pth


mkdir: cannot create directory ‘checkpoints’: File exists
--2023-06-30 08:30:50--  https://download.openmmlab.com/mmdetection/v3.0/efficientdet/efficientdet_effb3_bifpn_8xb16-crop896-300e_coco/efficientdet_effb3_bifpn_8xb16-crop896-300e_coco_20230223_122457-e6f7a833.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 163.181.81.214, 163.181.81.217, 163.181.81.213, ...
Connecting to download.openmmlab.com (download.openmmlab.com)|163.181.81.214|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



- configuration 수정

In [ ]:
from mmengine.runner import set_random_seed

# Modify dataset classes and color
# https://github.com/open-mmlab/mmdetection/issues/9610
cfg.metainfo = {
  'classes': ('65621', '50098', '30152', '45219', '30064',
              '30166', '50117', '50062', '30120', '20211',
              '10178', '45221', '10092', '30061', '10091',
              '30119', '25679', '30086', '50063', '15033',
              '45222', '65629', '65858', '30066', '50061',
              '20164', '10094', '90078', '30140', '15046',
              '45227', '30060', '15175', '10093', '65723',
              '90072', '10210', '30292', '10209', '45220',
              '35044', '30099', '30096', '65727', '65719',
              '65890', '90073', '10095', '20167', '30291'),
    'palette': [(220, 20, 60),(220, 20, 60),(220, 20, 60),(220, 20, 60),(220, 20, 60),
                (220, 20, 60),(220, 20, 60),(220, 20, 60),(220, 20, 60),(220, 20, 60),
                (220, 20, 60),(220, 20, 60),(220, 20, 60),(220, 20, 60),(220, 20, 60),
                (220, 20, 60),(220, 20, 60),(220, 20, 60),(220, 20, 60),(220, 20, 60),
                (220, 20, 60),(220, 20, 60),(220, 20, 60),(220, 20, 60),(220, 20, 60),
                (220, 20, 60),(220, 20, 60),(220, 20, 60),(220, 20, 60),(220, 20, 60),
                (220, 20, 60),(220, 20, 60),(220, 20, 60),(220, 20, 60),(220, 20, 60),
                (220, 20, 60),(220, 20, 60),(220, 20, 60),(220, 20, 60),(220, 20, 60),
                (220, 20, 60),(220, 20, 60),(220, 20, 60),(220, 20, 60),(220, 20, 60),
                (220, 20, 60),(220, 20, 60),(220, 20, 60),(220, 20, 60),(220, 20, 60)]
}


cfg.train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=20, val_interval=4)
cfg.train_dataloader.batch_size = 4

# Modify dataset type and path
cfg.data_root = '/content/snack-data-to50' #'./ballondatasets/balloon'

cfg.train_dataloader.dataset.ann_file = 'train/train.json' #'train/annotation_coco.json'
cfg.train_dataloader.dataset.data_root = cfg.data_root
cfg.train_dataloader.dataset.data_prefix.img = 'train/images'
cfg.train_dataloader.dataset.metainfo = cfg.metainfo

cfg.val_dataloader.dataset.ann_file = 'valid/valid.json'
cfg.val_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.data_prefix.img = 'valid/images'
cfg.val_dataloader.dataset.metainfo = cfg.metainfo

cfg.test_dataloader = cfg.val_dataloader


# Modify metric config
cfg.val_evaluator.ann_file = cfg.data_root+'/'+ 'valid/valid.json'  #'val/annotation_coco.json'
cfg.test_evaluator = cfg.val_evaluator

# Modify num classes of the model in box head and mask head
cfg.model.bbox_head.num_classes = 50                                #cfg.model.roi_head.bbox_head.num_classes = 1
                                                                    #cfg.model.roi_head.mask_head.num_classes = 1

# We can still the pre-trained Mask RCNN model to obtain a higher performance
cfg.load_from = 'checkpoints/efficientdet_training.pth'

# Set up working dir to save files and logs.
cfg.work_dir = '/content/drive/MyDrive'


# We can set the evaluation interval to reduce the evaluation times
cfg.train_cfg.val_interval = 4
# We can set the checkpoint saving interval to reduce the storage cost
cfg.default_hooks.checkpoint.interval = 5

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optim_wrapper.optimizer.lr = 0.02 / 8
cfg.default_hooks.logger.interval = 5


# Set seed thus the results are more reproducible
# cfg.seed = 0
set_random_seed(0, deterministic=False)

# We can also use tensorboard to log the training process
# cfg.visualizer.vis_backends.append({"type":'TensorboardVisBackend'})

print(f'Config:\n{cfg.pretty_text}')

Config:
dataset_type = 'CocoDataset'
data_root = '/content/snack-data-to50'
backend_args = None
train_pipeline = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='RandomResize',
        scale=(896, 896),
        ratio_range=(0.1, 2.0),
        keep_ratio=True),
    dict(type='RandomCrop', crop_size=(896, 896)),
    dict(type='RandomFlip', prob=0.5),
    dict(type='PackDetInputs')
]
test_pipeline = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(type='Resize', scale=(896, 896), keep_ratio=True),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='PackDetInputs',
        meta_keys=('img_id', 'img_path', 'ori_shape', 'img_shape',
                   'scale_factor'))
]
train_dataloader = dict(
    batch_size=4,
    num_workers=8,
    persistent_workers=True,
    sampler=dict(type='DefaultSampler', shuffle=True, _scope_='mmdet'),
    batch_sampler=dict(type='AspectRatioBat

- 데이터 로드 및 학습 시작

In [ ]:
from mmengine.config import Config
from mmengine.runner import Runner
runner = Runner.from_cfg(cfg)

In [ ]:
runner.train()

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
06/30 03:50:59 - mmengine - INFO - paramwise_options -- backbone.layers.0.bn.weight:weight_decay=0.0
06/30 03:50:59 - mmengine - INFO - paramwise_options -- backbone.layers.0.bn.bias:weight_decay=0.0
06/30 03:50:59 - mmengine - INFO - paramwise_options -- backbone.layers.1.0.depthwise_conv.bn.weight:weight_decay=0.0
06/30 03:50:59 - mmengine - INFO - paramwise_options -- backbone.layers.1.0.depthwise_conv.bn.bias:weight_decay=0.0
06/30 03:50:59 - mmengine - INFO - paramwise_options -- backbone.layers.1.0.se.conv1.conv.bias:weight_decay=0.0
06/30 03:50:59 - mmengine - INFO - paramwise_options -- backbone.layers.1.0.se.conv2.conv.bias:weight_decay=0.0
06/30 03:50:59 - mmengine - INFO - paramwise_options -- backbone.layers.1.0.linear_conv.bn.weight:weight_decay=0.0
06/30 03:50:59 - mmengine - INFO - paramwise_options -- backbone.layers.1.0.linear_conv.bn.bias:weight_decay=0.0
06/30 03:50:59 - mmengine - INF

Downloading: "https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b3_3rdparty_8xb32-aa-advprop_in1k_20220119-53b41118.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b3_3rdparty_8xb32-aa-advprop_in1k_20220119-53b41118.pth


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
06/30 04:35:39 - mmengine - INFO - Epoch(train)  [6][  55/1425]  lr: 2.4989e-03  eta: 2:08:56  time: 0.3651  data_time: 0.0101  memory: 18521  grad_norm: 4.6145  loss: 0.3102  loss_cls: 0.2816  loss_bbox: 0.0287
06/30 04:35:41 - mmengine - INFO - Epoch(train)  [6][  60/1425]  lr: 2.4989e-03  eta: 2:08:54  time: 0.3650  data_time: 0.0100  memory: 18521  grad_norm: 4.5401  loss: 0.3096  loss_cls: 0.2800  loss_bbox: 0.0296
06/30 04:35:43 - mmengine - INFO - Epoch(train)  [6][  65/1425]  lr: 2.4989e-03  eta: 2:08:53  time: 0.3657  data_time: 0.0104  memory: 18521  grad_norm: 4.4638  loss: 0.3076  loss_cls: 0.2780  loss_bbox: 0.0296
06/30 04:35:45 - mmengine - INFO - Epoch(train)  [6][  70/1425]  lr: 2.4989e-03  eta: 2:08:51  time: 0.3654  data_time: 0.0105  memory: 18521  grad_norm: 4.3715  loss: 0.3138  loss_cls: 0.2836  loss_bbox: 0.0302
06/30 04:35:46 - mmengine - INFO - Epoch(train)  [6][  75/1425]  lr: 2.4989e-03  eta: 2:08:49  time: 0.3654  data_ti

EfficientDet(
  (data_preprocessor): DetDataPreprocessor(
    (batch_augments): ModuleList(
      (0): BatchFixedSizePad()
    )
  )
  (backbone): EfficientNet(
    (layers): ModuleList(
      (0): ConvModule(
        (conv): Conv2dSamePadding(3, 40, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (bn): _BatchNormXd(40, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (activate): Swish()
      )
      (1): Sequential(
        (0): InvertedResidual(
          (drop_path): Identity()
          (depthwise_conv): ConvModule(
            (conv): Conv2dSamePadding(40, 40, kernel_size=(3, 3), stride=(1, 1), groups=40, bias=False)
            (bn): _BatchNormXd(40, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (activate): Swish()
          )
          (se): SELayer(
            (global_avgpool): AdaptiveAvgPool2d(output_size=1)
            (conv1): ConvModule(
              (conv): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
 

# 훈련 결과 확인

In [ ]:
import mmcv
from mmdet.apis import init_detector, inference_detector
img = mmcv.imread('/content/IMG_3041.jpg',channel_order='rgb')
checkpoint_file = '/content/drive/MyDrive/epoch_20.pth'
model = init_detector(cfg, checkpoint_file, device='cpu')
new_result = inference_detector(model, img)
print(new_result)

In [ ]:
from mmdet.registry import VISUALIZERS
# init visualizer(run the block only once in jupyter notebook)
visualizer = VISUALIZERS.build(model.cfg.visualizer)
# the dataset_meta is loaded from the checkpoint and
# then pass to the model in init_detector
visualizer.dataset_meta = model.dataset_meta

In [ ]:
# show the results
visualizer.add_datasample(
    'result',
    img,
    data_sample=new_result,
    draw_gt = None,
    wait_time=0,
    pred_score_thr=0.8
)
visualizer.show()

In [ ]:
from mmengine.visualization import Visualizer

# get built visualizer
visualizer_now = Visualizer.get_current_instance()
#visualizer_now = Visualizer.get_instance()
# the dataset_meta is loaded from the checkpoint and
# then pass to the model in init_detector
visualizer_now.dataset_meta = model.dataset_meta
# show the results
visualizer_now.add_datasample(
    'new_result',
    img,
    data_sample=new_result,
    draw_gt= False,
    wait_time=0,
    out_file=None,
    step = 10,
    pred_score_thr=0.5
)
visualizer_now.show()